In [11]:
import requests 
import json
import csv
import time
from time import sleep
import pandas as pd
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# 設定 User-Agent

headers = {
    'cookie': 'over18=1', 
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

# 發送 GET 請求取得文章內容
url = "https://www.momoshop.com.tw/category/LgrpCategory.jsp?l_code=4300100000&sourcePageType=4"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

if response.status_code == 200:
    print(f'請求成功：{response.status_code}')
else:
    print(f'請求失敗：{response.status_code}')

請求成功：200


In [30]:
# 找到 bt_category_Content 的內容
content = soup.select_one('#bt_category_Content')

# 找到內容中的所有連結
links = content.find_all('a', href=True)

# 建立 CSV 檔案並寫入連結資料
with open('laptop_links.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['文字', '網址'])  # 寫入表頭

    # 遍歷連結並寫入 CSV 檔案
    for link in links:
        text = link.text.strip()  # 獲取連結文字
        href = link['href']  # 獲取連結網址
        
        if href.startswith('javascript:ShowMore'):
            continue
            
        # 如果網址前方不含 https://www.momoshop.com.tw，則加上
        if not href.startswith('https://www.momoshop.com.tw'):
            href = urljoin('https://www.momoshop.com.tw', href)

        writer.writerow([text, href])  # 寫入一行資料

In [29]:
# 讀取 laptop_links.csv 檔案，並取得網址
links = []
with open('laptop_links.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # 跳過表頭
    for row in reader:
        text, href = row
        links.append(href)


# 儲存符合條件的連結至 laptop_all_links.csv
with open('laptop_all_links.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['網址'])  # 寫入表頭

    # 遍歷連結，取得符合條件的連結並寫入 CSV 檔案
    for link in links:
        # 發起請求並獲取網頁內容
        
        response = requests.get(link)
        html = response.content
        time.sleep(5)
        # 使用 Beautiful Soup 解析網頁內容
        soup = BeautifulSoup(html, 'html.parser')

        # 找到 bt_2_layout_b1 內容
        content = soup.select_one('#bt_2_layout_b1')

        # 找到符合條件的連結
        target_links = content.select('#bt_2_112_1_L_1 > a[href^="/goods/GoodsDetail.jsp"]')
        time.sleep(5)
        # 寫入符合條件的連結至 CSV 檔案
        for target_link in target_links:
            href = target_link['href']
            full_link = urljoin('https://www.momoshop.com.tw', href)
            writer.writerow([full_link])  # 寫入一行資料


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))